In [ ]:
import tess_cpm
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astroquery.mast import Tesscut
from astropy.coordinates import SkyCoord
import lightkurve as lk

In [ ]:
plt.rcParams['figure.figsize'] = (14, 8)

In [ ]:
# fits_file = "tpf/tess-s0001-4-1_64.525833_-63.615669_64x64_astrocut.fits"
fits_file = "provided_sources/tess-s0015-1-1_322.878520_47.920650_64x64_astrocut.fits"
# fits_file = "tess-s0002-4-2_83.272000_-69.508733_64x64_astrocut.fits"
# tabby14_file = "ffi/tabby/tess-s0014-2-4_301.564386_44.456886_100x100_astrocut.fits"
tabby15_file = "ffi/tabby/tess-s0015-2-3_301.564386_44.456886_100x100_astrocut.fits"
# tau_ceti_file = "ffi/tau_ceti/tess-s0003-1-2_26.017048_-15.937482_100x100_astrocut.fits"
sn_file = "supernovae/tess-s0001-4-1_64.525833_-63.615669_100x100_astrocut.fits"
fits_file = "provided_sources/tess-s0015-1-1_322.878520_47.920650_64x64_astrocut.fits"

In [ ]:
tabby15 = tess_cpm.Source(tabby15_file)
tabby15.plot_cutout()

In [ ]:
tabby15.plot_cutout(rowrange=[48, 53], colrange=[48, 53])

In [ ]:
tabby15.set_aperture(rowrange=[49, 52], colrange=[49, 52])
tabby15.plot_cutout(rowrange=[47, 53], colrange=[48, 53], show_aperture=True)

In [ ]:
tabby15.plot_pix_by_pix()

In [ ]:
tabby15.add_cpm_model(predictor_method="similar_brightness")
tabby15.set_regs([0.01])
times, fluxes, predictions = tabby15.holdout_fit_predict(k=10)

In [ ]:
# tabby15.plot_pix_by_pix(data_type="raw")

In [ ]:
tabby15.plot_pix_by_pix(data_type="cpm_subtracted_lc", split=True)

In [ ]:
detrended_aperture_lc = tabby15.get_aperture_lc(data_type="cpm_subtracted_lc")

In [ ]:
plt.plot(tabby15.time, detrended_aperture_lc, "-", c="k")

In [ ]:
tabby15.set_aperture(rowrange=[49, 51], colrange=[50, 52])
tabby15.plot_cutout(rowrange=[47, 53], colrange=[48, 54], show_aperture=True)

In [ ]:
tabby15.add_cpm_model()
tabby15.set_regs([0.01])
times, fluxes, predictions = tabby15.holdout_fit_predict(k=2)

In [ ]:
tabby15.plot_pix_by_pix(data_type="cpm_subtracted_lc")

In [ ]:
detrended_apt_lc = tabby15.get_aperture_lc(data_type="cpm_subtracted_lc")
plt.plot(tabby15.time, detrended_apt_lc, "-", c="k")
lc = lk.TessLightCurve(time=tabby15.time, flux=detrended_apt_lc+1)
# lc.plot()
plt.plot(tabby15.time, lc.flatten().flux)

In [ ]:
cpm_regs = 10.0 ** np.arange(-12, 10)
min_reg, cdpps = tabby15.calc_min_cpm_reg(cpm_regs, k=10)
print(f"Min CDPP Reg: {min_reg}")

In [ ]:
tabby15.set_regs([0.00])
tabby15.holdout_fit_predict(k=tabby15.time.size);
flux_splits = tabby15.get_aperture_lc(data_type="raw", split=True)
splits = tabby15.get_aperture_lc(data_type="cpm_subtracted_lc", split=True)
f, axs = plt.subplots(2, 1, sharex=True, figsize=(12, 8))
for time, split, f_split in zip(tabby15.split_times, splits, flux_splits):
    axs[0].plot(time, split, ".")
    axs[1].plot(time, f_split, ".")

In [ ]:
f, axs = plt.subplots(2, 1, figsize=(12, 8))
for t, s, f_split in zip(tabby15.split_times, splits, flux_splits):
    axs[0].plot(t, s, ".")
    axs[1].plot(t, f_split, ".")

In [ ]:
for t, p, f in zip(tabby15.split_times, splits, flux_splits):
#     plt.plot(t, f, ".")
    plt.plot(t, p, ".")

In [ ]:
# tabby15.rescale()
tabby15.plot_pix_by_pix(data_type="cpm_subtracted_lc")
loo_lc = tabby15.get_aperture_lc(data_type="cpm_subtracted_lc")
plt.plot(tabby15.time, loo_lc, "-", c="k")

In [ ]:
n = tess_cpm.Source(fits_file)
n.plot_cutout()

In [ ]:
n.set_aperture(rowrange=[29, 32],colrange=[31, 32])
n.plot_cutout(rowrange=[29, 36], colrange=[29, 36], show_aperture=True)

In [ ]:
n.plot_pix_by_pix()

In [ ]:
n.add_cpm_model()
n.set_regs([0.5])
n.holdout_fit_predict(k=10);

In [ ]:
n.plot_pix_by_pix(data_type="cpm_subtracted_lc", split=True)

In [ ]:
detrended_lc = n.get_aperture_lc(data_type="cpm_subtracted_lc")

In [ ]:
plt.plot(n.time, detrended_lc, "-", c="k")

In [ ]:
lc = lk.TessLightCurve(time=n.time, flux=detrended_lc)
lc.plot()

In [ ]:
pg = lc.to_periodogram(oversample_factor=100)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15, 8))
pg.plot(ax=axs[0], c='k')
pg.plot(ax=axs[1], c='k', view='period')
fig.suptitle("Periodogram", fontsize=20, y=0.95)
period = pg.period_at_max_power
print(f"Max Power Period: {period}")
lc.fold(period.value*2).scatter()
plt.title(f"Folded Lightcurve with Period: {period:.4f}", fontsize=20);